Traitement de données massives avec Pyspark
========
Introduction à PySpark 
--------
- - - -

La structure RDD que nous avons étudiée est importante. Cependant, lorsqu'il s'agit de gérer des bases de données structurées (organisées en colonnes), nous verrons qu'il existe une structure plus optimale. Les RDDs sont néanmoins extrêmement performants devant un problème non structuré tel qu'un texte.

Dans cet exercice, nous allons utiliser la structure RDD pour une base de données non structurée : l'intégrale des Misérables de Victor Hugo. L'objectif est de construire un simple WordCount comptant le nombre d'occurrences de chaque mot, pour approfondir et mettre en pratique vos compétences en RDD et map reduce.

Pour cet exercice, nous considérons un fichier texte contenant l'intégralité des Misérables de Victor Hugo.

## 1. Création d'un SparkContext

In [1]:
# Import de SparkContext du module pyspark
from pyspark import SparkContext

# Défintion d'un SparkContext
sc = SparkContext.getOrCreate()

## 2. Importation de la base de données

* (a) Importer le fichier miserables_full.txt dans un RDD nommé miserables.<br>
* (b) Afficher les 10 premières lignes du RDD.

In [2]:
# Chargement du fichier "miserables_full.txt" et affichage des 10 premières lignes
miserables = sc.textFile("data/miserables_tome1.txt")
miserables.take(10)

['The Project Gutenberg EBook of Les misérables Tome I, by Victor Hugo',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 'Title: Les misérables Tome I',
 '       Fantine']

## 3. Mise en forme de la base

* Pour compter le nombre d'occurrences d'un mot, il faut mettre tous les mots en forme de la même façon :
    * Mettre toutes les lettres en minuscule grâce à la méthode lower des chaînes des caractères.
    * Remplacer la ponctuation collée aux mots ( .  ,  -  ' ) par des espaces grâce à la méthode replace des chaînes de caractères, qui s'applique de la façon suivante : <code>str.replace(',', ' ')</code><br>


* (a) Créer un RDD nommé miserables_clean, qui contient le texte des Misérables en minuscule et sans ponctuation à l'aide des méthodes map, lower et replace.

In [3]:
# Création d'un RDD nettoyé
miserables_clean = miserables.map(lambda x : x.lower().replace(',', ' ').replace('.', ' ').replace('-', ' ').replace('’', ' '))

Maintenant que tous les mots sont bien séparés par des espaces, il est possible utiliser la méthode split pour séparer les mots de chaque ligne.

Il est possible d'utiliser la méthode map mais le problème ici sera que map crée une liste de listes.

En effet, un split transforme chaque ligne en liste et la base de données étant une liste de lignes, le résultat final sera une liste de listes. Dans ce cas, il est coûteux de comparer un même mot sur deux lignes différentes car ils sont tous les deux dans des listes différentes.

Pour pallier ce problème, il existe une méthode flatMap, qui s'utilise exactement de la même façon que map, à la différence que le résultat final est un RDD à une seule dimension.

* (b) Créer un RDD nommé miserables_flat qui contient l'ensemble des mots sous une seule dimension.

In [4]:
# Création d'un RDD séparant les mots
miserables_flat = miserables_clean.flatMap(lambda line: line.split(" "))
miserables_flat.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'les',
 'misérables',
 'tome',
 'i',
 '']

# 4. Map-Reduce

* (a) À partir de miserables_flat, créer un RDD mots contenant l'ensemble des couples (mot, nb_occurences) à l'aide des méthodes map et reduceByKey.

In [5]:
# création d'un RDD contenant l'ensemble des couples (mot, nb_occurences) 
mots = miserables_flat.map(lambda x : (x,1)) \
                      .reduceByKey(lambda x,y : x + y)

<div class="alert alert-info" role="alert">
Pour compter le nombre d'occurrences d'un élément, une technique consiste à :
- Utiliser la méthode map pour créer un couple clé/valeur où chaque mot est une clé, chaque valeur vaut 1.
- Utiliser reduceByKey pour additionner les valeurs pour chaque mot.
</div>

* (b) Créer une liste contenant les couples (mot, occurrence) ordonnée dans l'ordre croissant des occurrences.

In [6]:
### Première méthode de tri

# Tri en utilisant la fonction 'sorted' des RDD
mots_sorted  = sorted(mots.collect(),
                     key= lambda x: x[1],
                     reverse= 0)

### Deuxième méthode de tri

# Tri en utilisant la fonction 'sortBy' des RDD puis convertir en liste en utilisant collect
mots_sorted_2 = mots.sortBy(lambda couple: couple[1], ascending = True) \
                    .collect()

## 5. Succession de méthodes

In [8]:
# Création d'une liste à partir du fichier texte
mots_sorted_3 = sc.textFile("data/miserables_tome1.txt") \
                  .map(lambda x : x.lower().replace(',', ' ').replace('.', ' ').replace('-', ' ').replace('’', ' ')) \
                  .flatMap(lambda line: line.split(" ")) \
                  .map(lambda x : (x,1)) \
                  .reduceByKey(lambda x,y : x + y) \
                  .sortBy(lambda couple: couple[1], ascending = True) \
                  .collect()
                
print(mots_sorted_3[:10])

[('author:', 1), ('date:', 1), ('january', 1), ('last', 1), ('july', 1), ('language:', 1), ('parlement', 1), ("d'aix;", 1), ('hériter', 1), ('usage', 1)]


* __(b)__ Fermer le SparkContext sc.

In [9]:
# Fermeture du SparkContext
sc.stop()